# Conversation Knowledge Graph
This type of memory uses a knowledge graph to recreate memory.

## Using memory with LLM

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain_openai import OpenAI

In [ ]:
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [ ]:
memory.load_memory_variables({"input": "who is sam"})

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [ ]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [ ]:
memory.load_memory_variables({"input": "who is sam"})

In [ ]:
memory.get_current_entities("what's Sams favorite color?")

We can also more modularly get knowledge triplets from a new message (will use previous messages as context).

In [ ]:
memory.get_knowledge_triplets("her favorite color is red")

## Using in a chain

In [ ]:
llm = OpenAI(temperature=0)
from langchain.chains.conversation.base import ConversationChain
from langchain_core.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm,
    # verbose=True,
    prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [ ]:
conversation_with_kg.predict(input="Hi, what's up?")

In [ ]:
conversation_with_kg.predict(
    input="My name is James and I'm helping Will. He's an engineer."
)

In [ ]:
conversation_with_kg.predict(input="What do you know about Will?")